In [ ]:
import torch
import torch.nn as nn

class FizBuzNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FizBuzNet, self).__init__()
        self.hidden = nn.Linear(input_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, batch):
        hidden = self.hidden(batch)
        activated = torch.sigmoid(hidden)
        out = self.out(activated)
        return out

1.nn.Module

可以将网络的每个可分离组件定义为继承自nn.Module的单独Python类。
它实现了两个主要函数，__call__()与backward()，
用户需要重写forward()和__init__()。

Linear类中，用Parameter封装张量。Parameter将权重和偏置添加到模块参数列表中，
并且在调用model.Parameters()时可用。

（1）apply

apply(fn)：将fn函数递归地应用到网络模型的每个子模型中，主要用在参数的初始化。

使用apply()时，需要先定义一个参数初始化的函数。

In [ ]:
def weight_init(m):
    classname = m.__class__.__name__ # 得到网络层的名字，如ConvTranspose2d
    if classname.find('Conv') != -1:  # 使用了find函数，如果不存在返回值为-1，所以让其不等于-1
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

之后，定义自己的网络，得到网络模型，使用apply()函数，就可以分别对conv层和bn层进行参数初始化。

In [ ]:
model = net()
model.apply(weight_init)

（2）train和eval

model.train()和model.eval()的区别主要在于Batch Normalization和Dropout两层。

如果模型中有BN层(Batch Normalization）和 Dropout，需要在训练时添加model.train()。
model.train()是保证BN层能够用到每一批数据的均值和方差。
对于Dropout，model.train()是随机取一部分网络连接来训练更新参数。

如果模型中有BN层(Batch Normalization）和Dropout，在测试时添加model.eval()。
model.eval()是保证BN层能够用全部训练数据的均值和方差，即测试过程中要保证BN层的均值和方差不变。
对于Dropout，model.eval()是利用到了所有网络连接，即不进行随机舍弃神经元。

训练完train样本后，生成的模型model要用来测试样本。
在model(test)之前，需要加上model.eval()，否则的话，有输入数据，即使不训练，它也会改变权值。
这是model中含有BN层和Dropout所带来的的性质。

在做one classification的时候，训练集和测试集的样本分布是不一样的，尤其需要注意这一点。

（3）parameters

调用parameters会返回所有模型参数。对于优化器或使用参数进行试验非常重要。
无需逐行写下每个参数更新，我们可以将其归纳为for循环。

In [ ]:
net = FizBuzNet(input_size, hidden_size, output_size)

with torch.no_grad():
    for p in net.parameters():
        p -= p.grad * lr

（4）zero_grad

这是使梯度变为0的便捷函数。不必查找每个参数分别调用。
对模型对象的一次调用将使得所有参数的梯度变成0。

2.nn.function

nn.functional是一个很常用的模块，nn中的大多数layer在functional中都有一个与之对应的函数。
nn.functional中的函数与nn.Module()的区别是：

nn.Module实现的层（layer）是一个特殊的类，都是由class Layer(nn.Module)定义，
会自动提取可学习的参数。
nn.functional中的函数更像是纯函数,由def functional(input）定义。

注意：

如果模型有可学习的参数时，最好使用nn.Module；
否则既可以使用nn.functional也可以使用nn.Module，二者在性能上没有太大差异，
具体的使用方式取决于个人喜好。
由于激活函数（ReLu、sigmoid、Tanh）、池化（MaxPool）等层没有可学习的参数，
可以使用对应的functional函数，而卷积、全连接等有可学习参数的网络建议使用nn.Module。

注意：

虽然dropout没有可学习参数，但建议还是使用nn.Dropout而不是nn.functional.dropout，
因为dropout在训练和测试两个阶段的行为有所差别，
使用nn.Module对象能够通过model.eval操作加以区分。

在代码中，不具备可学习参数的层（激活层、池化层），将它们用函数代替，
这样可以不用放置在构造函数__init__中。有可学习的模块，也可以用functional代替，
只不过实现起来比较繁琐，需要手动定义参数parameter，如前面实现自定义的全连接层，
就可以将weight和bias两个参数单独拿出来，在构造函数中初始化为parameter。